In [1]:
import sys
sys.path.append('../')

from lxml import etree
import abc
import re
import requests
import pandas
import scipy
from pprint import pprint

from gensim.models.phrases import *

import gensim.models as models
import gensim.models.word2vec as word2vec
import secure

import searchbetter.search as search
reload(search)
import searchbetter.rewriter as rewriter
reload(rewriter)


<module 'searchbetter.rewriter' from '../searchbetter/rewriter.pyc'>

In [2]:
import searchbetter.rewriter as rewriter 
reload(rewriter)
import secure

model_path = secure.MODEL_PATH_BASE+'word2vec/word2vec'
w2v_rewriter = rewriter.Word2VecRewriter(model_path, create=False)

In [3]:
w2v_rewriter.rewrite("New York")

[u'philadelphia',
 u'los angeles',
 u'london',
 u'boston',
 u'chicago',
 u'princeton',
 u'seattle',
 u'chicago illinois',
 u'san francisco',
 u'ny',
 u'New York']

In [4]:


import searchbetter.search as search
reload(search)

import secure

# Create a search engine that searches over all edX courses.
# Under the hood, this uses Python's Whoosh library to index
# the course data stored in a CSV and then run searches against it.
dataset_path = secure.DATASET_PATH_BASE+'Master CourseListings - edX.csv'
index_path = secure.INDEX_PATH_BASE+'edx'
edx_engine = search.EdXSearchEngine(dataset_path, index_path, create=False)

In [5]:
edx_engine.search("chemistry")

[{'course_id': u'DavidsonX/D.001x/3T2014', 'name': u'Medicinal Chemistry'},
 {'course_id': u'course-v1:DavidsonX+D001x+3T2015',
  'name': u'Medicinal Chemistry'},
 {'course_id': u'course-v1:CooperUnion+Chem.1x+2T2016',
  'name': u'Preparing for CLEP Chemistry: Part 1'},
 {'course_id': u'course-v1:CooperUnion+Chem.2x+2T2016',
  'name': u'Preparing for CLEP Chemistry: Part 2'},
 {'course_id': u'MITx/3.091x_2/1T2014',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'MITx/3.091x_3/3T2014',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'course-v1:MITx+3.091x_4+1T2015',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'course-v1:MITx+3.091x_5+3T2015',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'course-v1:MITx+3.091x+3T2016',
  'name': u'Introduction to Solid State Chemistry'},
 {'course_id': u'CooperUnion/Chem.1x/1T2015',
  'name': u'Preparing for the AP* Chemistry Exam - Part 1'}]

## Testing

In [6]:
# comparison testingt
# TODO load from file
search_terms = [
    "chemical reaction",
    "psychology",
    "cognitive bias",
    "machine learning"
]

rewriters = [
    rewriter.ControlRewriter(),
    rewriter.WikipediaRewriter(),
    w2v_rewriter
]

In [ ]:
def stats(term):
    ans = [num_results(term, rw) for rw in rewriters]
    ans = [term] + ans
    return ans
    
    
def num_results(term, rw):
    edx_engine.set_rewriter(rw)
    results = edx_engine.search(term)
    num_results = len(results)
    return num_results

with open('../test/test-search-terms/generic.txt', 'r') as f:
    # read terms but chop the newlines at the end of each line
    terms = [line.rstrip('\n') for line in f]
    data = [stats(term) for term in terms]

df = pandas.DataFrame(columns=["term","control","wiki","word2vec"],data=data)

In [ ]:
import plotly
import plotly.graph_objs as go
import plotly.offline as py

py.init_notebook_mode()

In [ ]:
df

In [ ]:
xs = list(df['control'])

In [ ]:
ys = list(df['wiki'])

In [ ]:
y2 = list(df['word2vec'])

In [ ]:
import webcolors
# colors plotly uses

colors = [
    '#1f77b4', # blue
    '#ff7f0e', # orange
    '#2ca02c', # green
    '#d62728', # red
    '#9467bd'  # purple
]

# need to convert, e.g., #FF0000 to 'rgb(255,0,0)'
rgb_colors = [webcolors.hex_to_rgb(color) for color in colors]
color_strings = ['rgb(%s,%s,%s)' % (c[0], c[1], c[2]) for c in rgb_colors]

In [ ]:
# reference: https://plot.ly/python/reference/#scattergl

traceControl = go.Scattergl(
    x = xs,
    y = xs,
    mode = 'lines',
    name = 'Control (no rewriting)',
    hoverinfo = 'text+name',
#     opacity = 0.3,
    line = dict(
#         color = 'rgb(0,0,0)',
#         opacity = 0.3
        color = color_strings[0]
    )
)

traceWiki = go.Scattergl(
    x = xs,
    y = ys,
    mode = 'markers',
    name = 'Wiki',
    marker = dict(
        color = color_strings[1],
        opacity = 0.5,
        size = 12,
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
#         width = 5

    )
)

traceWikiLOBF = go.Scattergl(
    x = np.unique(xs),
    y = np.poly1d(np.polyfit(xs, ys, 1))(np.unique(xs)),
    mode = 'lines',
    name = 'WikiLOBF',
    hoverinfo = 'text+name',
    line = dict(
        color = color_strings[1]
    )
)

traceWord2Vec = go.Scattergl(
    x = xs,
    y = y2,
    mode = 'markers',
    name = 'Word2Vec',
    marker = dict(
        color = color_strings[2],
        opacity = 0.5,
        size = 12,
        line = dict(
            width = 2,
            color = 'rgb(0, 0, 0)'
        )
#         width = 5
#         color = 'rgba(255,0,0,0.2)'
    )
)

traceWord2VecLOBF = go.Scattergl(
    x = np.unique(xs),
    y = np.poly1d(np.polyfit(xs, y2, 1))(np.unique(xs)),
    mode = 'lines',
    name = 'Word2VecLOBF',
    hoverinfo = 'text+name',
    line = dict(
        color = color_strings[2]
    )
)


def plotSeriesWithRegression(xs, ys, name, color):
    # setting legend group lets user toggle the series AND its
    # line of best fit together
    legendgroup_name = '%s group' % name
    
    # plot points
    traceScatter = go.Scattergl(
        x = xs,
        y = ys,
        mode = 'markers',
        name = name,
        legendgroup = legendgroup_name,
        marker = dict(
            color = color,
            opacity = 0.5,
            size = 12,
            line = dict(
                width = 2,
                color = 'rgb(0, 0, 0)'
            )
        )
    )
    
    # plot line of best fit
    slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(xs, ys)
    line_of_best_fit = np.poly1d([slope, intercept])
    r_squared = r_value ** 2
    
    # to show line of best fit
    # y = %.2fx + %.2f, r^2 = %.2f' % (slope, intercept, r_squared),
    
    traceRegression = go.Scattergl(
        x = np.unique(xs),
        y = line_of_best_fit(np.unique(xs)),
        mode = 'lines',
        name = name + ' linear regression',
        legendgroup = legendgroup_name,
        hoverinfo = 'text+name',
        line = dict(
            color = color
        )
    )
    
    return [traceScatter, traceRegression]


# plot wiki
wikiTraces = plotSeriesWithRegression(xs, ys, name='Wikipedia Categories', color=color_strings[1])
w2vTraces = plotSeriesWithRegression(xs, y2, name='Word2Vec', color=color_strings[2])

plot = [traceControl] + wikiTraces + w2vTraces

layout = go.Layout(
    title='Effect of query rewriting on search engine hits (Udacity)',
    xaxis=dict(
        title='# hits before rewriting'
    ),
    yaxis=dict(
        title='# hits after rewriting'
    )
)

fig = go.Figure(data=plot, layout=layout)

# Plot and embed in ipython notebook!
py.iplot(fig)

In [ ]:
np.poly1d(np.polyfit(xs, ys, 1))

In [ ]:
    coeffs, residual, _, _, _ = np.polyfit(xs, ys, 1, full=True)

In [ ]:
coeffs

In [ ]:
residual

In [ ]:
slope, intercept, r_value, p_value, std_err = scipy.stats.linregress(xs, ys)

In [ ]:
slope

In [ ]:
intercept

In [ ]:
r_value

In [ ]:
p_value

In [ ]:
np.poly1d([slope, intercept])

In [ ]:
['rgb(%s,%s,%s)' % (c[0], c[1], c[2]) for c in rgb_colors]

In [ ]:
(1,2,3)[0]